# SOTA Spots Map
by www.operator-paramedyk.pl / SQ9NIL

explained step-by-step

## What SOTA is?
SOTA (Summits On The Air) is an activity designed for radio amateurs (called also HAMs) who like hiking. It is about communication via ration between an activator - operator who climbed a designated summit (map available at https://sotl.as/map) - and chasers - all other operators. Call (or QSO) may be done with the use of telegraphy (morse code), voice or data transmission on any band available for radio amateurs. To make a successfull call, both operators need to exchange report, which say how they hear each other, and log it into a log. SOTA results are then uploaded into https://www.sotadata.org.uk/en/ webpage.

If you hear someone activating a summit (or want to make chasers aware the you are currently activating), you can send a spot for dedicated webpage (https://sotawatch.sota.org.uk/en/).

You can find all information about SOTA programme at https://www.sota.org.uk/.

This script is designed for chasers who want to check which summits are currently activated. With the use of data of latest activations received via SOTA API and summits coordinates in SOTA Database file, it creates an interactive map to visualise frequency and band of each activation.

For best experience, you can run this script during weekends, when there's a spike in SOTA activity.

You may also find helpful live version of this script I deployed at my webpage: https://www.operator-paramedyk.pl/sota/.

## Script algorithm

### Import of relevant packages
There are seven packages to be installed for this script, all specified in requirements.txt file. You can install all of them at once typing in terminal following line:

```pip install -r requirements.txt```

If you use Google Colab (https://colab.research.google.com/), all of them should be installed by default, so you can import them directly.

In [1]:
import requests # for communication with API
import pandas as pd # for data analysis
from datetime import datetime, timedelta # for time calculations
import folium # for data visualisation on a map

### Bands and modes
Before the analysis of SOTA data, I'll create two dataframes to store bands and modes available for HAM radio amateurs. Less popular ones - below 1.8 MHz (wavelength > 180 m) and above 900 MHz (wavelength < 30 cm) will be clustered together. To each band and mode I'll assign a color to be used for further visualisation.

In [2]:
# create dataframes to store bands assign them to colors for visualisation
# lower and upper freqs does not refer exactly to bandplan to make sure frequencies are mapped correctly during visualisation
bands = {
    'band': ['1.8 MHz or below', '3.5 MHz', '5 MHz', '7 MHz', '10 MHz', '14 MHz', '18 MHz', '21 MHz', '24 MHz', '28 MHz', '50 MHz', '70 MHz', '144 MHz', '220 MHz', '433 MHz', '900 MHz or above'],
    'lower_freq': [0, 3, 4.5, 6, 9, 13, 16, 19, 24, 27, 45, 65, 142, 210, 420, 850],
    'upper_freq': [2.5, 4, 5.5, 8, 11, 15, 18.5, 23, 26, 35, 55, 75, 148, 240, 460, 500000],
    'color': ['saddlebrown','chocolate', 'brown','red', 'salmon', 'orange', 'darkkhaki', 'yellow', 'olivedrab', 'green', 'lime', 'cyan', 'blue', 'purple', 'magenta', 'pink'],
}
bands_df = pd.DataFrame(bands)

bands_df = bands_df.set_index('band')
bands_df['color'] = bands_df['color'].astype('string')

In [3]:
# create dataframes to store modes and assign them to colors for visualisation
modes = {
    'mode': ['AM', 'CW', 'Data', 'DV', 'FM', 'SSB', 'Other'],
    'color': ['lime', 'red', 'cyan', 'magenta', 'yellow', 'blue', 'orange']
}

modes_df = pd.DataFrame(modes)
modes_df['color'] = modes_df['color'].astype('string')
modes_df['mode'] = modes_df['mode'].astype('string')

### Get activations data

Now I'll create a function to download data on latest activations with the use of SOTA API. Argument used during function execution is used in API request to define spots in scope. If the number is **negative**, it defines the timerange (in hours) of spots to be downloaded (by default function looks for the spots sent in latest 1 hour). If it's **positive**, the asked number of latest spots will be provided.

If you use negative number and no spots will be found, function will download latest 10 spots to make sure there are data to be analysed.

Spots downloaded are converted via JSON format to a dictionary, which is returned by the function.

In [4]:
def get_spots(time = -1):
      """Downdload SOTA spots sent in defined timeframe or defined number of latests spots and returns them as dictionary"""
      # if time is negative - download spots alerted in defined number of alerts
      # if time is positive - download given number of latest spots
      # by default looking from a spots sent in last 1 hour
      temp_spots_dict = {}
      url = f'https://api2.sota.org.uk/api/spots/{time}/all'
      r = requests.get(url)
      print(f'Status code: {r.status_code}')
      temp_spots_dict = r.json()
      if time > 0:
            print(f'{len(temp_spots_dict)} found where expected number was {time}.')
      if time <= 0:
            print(f'{len(temp_spots_dict)} spots found in latest {-time} h.')
      # if there are no spots sent in time provided, return latest 10 to make sure dictionary is not empty
      if len(temp_spots_dict) == 0:
          temp_spots_dict = get_spots(10)
      return temp_spots_dict


Now we'll use this function to download the spots sent in last hour and convert the dictionary into DataFrame.

In [5]:
# import spots and convert them into DataFrame
spots_dict = get_spots(-1)

spots_df = pd.DataFrame(spots_dict)

Status code: 200
52 spots found in latest 1 h.


Single spot looks good:

In [6]:
spots_dict[0]

{'id': 658095,
 'userID': 0,
 'timeStamp': '2022-04-23T10:26:02.797',
 'comments': '[RBNHole] at DJ9IE 23 WPM 17 dB SNR',
 'callsign': 'RBNHOLE',
 'associationCode': 'EI',
 'summitCode': 'IW-026',
 'activatorCallsign': 'EI3KA/P',
 'activatorName': 'John',
 'frequency': '14.0660',
 'mode': 'CW',
 'summitDetails': 'Garraun, 598m, 6 pts',
 'highlightColor': 'red'}

but if you look at the datatypes in dataframe, all information is saved as object type.

In [7]:
spots_df.dtypes

id                    int64
userID                int64
timeStamp            object
comments             object
callsign             object
associationCode      object
summitCode           object
activatorCallsign    object
activatorName        object
frequency            object
mode                 object
summitDetails        object
highlightColor       object
dtype: object

To make furtther analysis possible, we need to convert the data to string, numeric or date format. Also, I'll create one more column to get full summit reference (country/region-summit ID), which we'll use later to match between spot and SOTA database.

After this, we can confirm all data in spots_df DataFrame has appropriate type.

In [8]:
# convert datatypes for relevant fields
spots_df['activatorCallsign'] = spots_df['activatorCallsign'].astype('string')
spots_df['associationCode'] = spots_df['associationCode'].astype('string')
spots_df['summitCode'] = spots_df['summitCode'].astype('string')
spots_df['mode'] = spots_df['mode'].astype('string')
spots_df['frequency'] = spots_df['frequency'].astype('float')
spots_df['timeStamp'] = pd.to_datetime(spots_df['timeStamp'])
# add full summit codes column to spots_df DataFrame
spots_df['summit'] = spots_df['associationCode']+'/'+spots_df['summitCode']

In [9]:
spots_df.dtypes

id                            int64
userID                        int64
timeStamp            datetime64[ns]
comments                     object
callsign                     object
associationCode              string
summitCode                   string
activatorCallsign            string
activatorName                object
frequency                   float64
mode                         string
summitDetails                object
highlightColor               object
summit                       string
dtype: object

And we can check data for all spots downloaded:

In [10]:
spots_df

,id,userID,timeStamp,comments,callsign,associationCode,summitCode,activatorCallsign,activatorName,frequency,mode,summitDetails,highlightColor,summit
0,658095,0,2022-04-23 10:26:02.797,[RBNHole] at DJ9IE 23 WPM 17 dB SNR,RBNHOLE,EI,IW-026,EI3KA/P,John,14.0660,CW,"Garraun, 598m, 6 pts",red,EI/IW-026
1,658094,0,2022-04-23 10:25:47.243,51 in Cardiff Kevin,GW5NIC,GW,MW-036,MW0KXN/P,Kevin,145.4500,FM,"Pen-crug-melyn, 326m, 1 pt",red,GW/MW-036
2,658093,0,2022-04-23 10:24:37.007,[SOTA Spotter],CT2GSN,CT,MN-049,CT2GSN/P,João,28.4610,ssb,"Santa Tecla, 317m, 1 pt",red,CT/MN-049
3,658092,0,2022-04-23 10:24:33.923,None,MM7MWL,GM,ES-034,MM7MWL/P,Mike,14.2730,SSB,"Mona Gowan, 749m, 4 pts",red,GM/ES-034
4,658091,0,2022-04-23 10:20:05.233,[SOTA Spotter],SQ9RNW,SP,WS-002,SQ9RNW/P,MIROSLAW,28.4500,ssb,"Góra Świętej Anny, 400m, 1 pt",red,SP/WS-002
5,658090,0,2022-04-23 10:19:02.923,None,G4OOE,G,TW-004,G4OOE/P,Nick,14.0610,CW,"Bishop Wilton Wold, 248m, 1 pt",red,G/TW-004
6,658089,0,2022-04-23 10:18:40.547,None,MW0KXN,GW,MW-036,MW0KXN/P,Kevin,145.4500,FM,"Pen-crug-melyn, 326m, 1 pt",red,GW/MW-036
7,658088,0,2022-04-23 10:16:01.990,[RBNHole] at OE9GHV 17 WPM 22 dB SNR,RBNHOLE,OE,SB-268,OE1WED/2,Gerhard,7.0229,CW,"Gaisberg, 1287m, 4 pts",red,OE/SB-268
8,658087,0,2022-04-23 10:13:26.200,None,CT2IXP,CT,MN-049,CT2GSN/P,João,145.5250,FM,"Santa Tecla, 317m, 1 pt",red,CT/MN-049
9,658086,0,2022-04-23 10:12:57.683,[SOTA Spotter],EI6FR,EI,IE-042,EI6FR/P,Declan,14.0640,cw,"Cushbawn, 400m, 4 pts",red,EI/IE-042


### Import data from SOTA Database

As you can see, spots do not provide details on activated summits, beside their reference codes. To retrieve this, we need to use SOTA Database. We can use SOTA API for this, but it will be time consuming, so I'll download CSV file with all the summits, stored at https://www.sotadata.org.uk/summitslist.csv. You can also use local file stored in repository - in this case you need to comment line 5 and uncomment line 6 in the cell below.

CSV file includes following columns:
- 0: Summit's reference code,
- 1: association name,
- 2: region's name,
- 3: summit's name,
- 4: altitude (in meters),
- 5: altitude (in feets),
- 6 and 8: longitude,
- 7 and 9: latitude,
- 10: points assigned to a summit in SOTA programme,
- 11: bonus points for activation in winter,
- 12: date when summit has been added to SOTA programme,
- 13: summit's expiry date in SOTA programme,
- 14: number of activations
- 15: last activation date,
- 16: last logged activator.

To make data usable, I need to convert it to proper format. Then, I'll re-index dataframe created so summits are indexed with their reference code.

In [11]:
# create DataFrame based on csv file with all the summits saved (regularly updated
# from https://www.sotadata.org.uk/summitslist.csv, and converting the datatypes
# first row of CSV file is a header, so should be ignored

SOTA_summits_df = pd.read_csv('https://www.sotadata.org.uk/summitslist.csv', skiprows = 1, dtype = {
#SOTA_summits_df = pd.read_csv('summitslist.csv', skiprows = 1, dtype = {
    0: 'string', # summit's reference code
    1: 'string', # association name
    2: 'string', # region name
    3: 'string', # summits name
    4: 'int', # altitude in meters
    5: 'int', # altitude in feets
    6: 'string', # longitude
    7: 'string', # latitude
    8: 'float', # longitude
    9: 'float', # latitude
    10: 'int', # SOTA points for summit
    11: 'int', # bonus points for activations in winter
#    12: not relevant - date when summit was added to SOTA
#    13: not relevant - summit's expiry date
    14: 'int', # number of activiations
#    15: not relevant - last activation date
#    16: not relevant - last activation logged
}
)

SOTA_summits_df = SOTA_summits_df.set_index('SummitCode')

In [12]:
SOTA_summits_df.head()

,AssociationName,RegionName,SummitName,AltM,AltFt,GridRef1,GridRef2,Longitude,Latitude,Points,BonusPoints,ValidFrom,ValidTo,ActivationCount,ActivationDate,ActivationCall
SummitCode,,,,,,,,,,,,,,,,
3Y/BV-001,Bouvet Island,Bouvetøya (Bouvet Island),Olavtoppen,780,2559,3.3565,-54.4104,3.3565,-54.4104,10,3,01/03/2018,31/12/2099,0,NaN,NaN
4O/IC-001,Montenegro,Istok Crne Gore,Maja Rosit,2524,8280,19.8505,42.4795,19.8505,42.4795,10,3,01/03/2019,31/12/2099,0,NaN,NaN
4O/IC-002,Montenegro,Istok Crne Gore,Kom kučki,2487,8159,19.6417,42.6807,19.6417,42.6807,10,3,01/03/2019,31/12/2099,0,NaN,NaN
4O/IC-003,Montenegro,Istok Crne Gore,Veliki vrh (Maja Gurt e Zjarmit),2480,8136,19.7872,42.4971,19.7872,42.4971,10,3,01/03/2019,31/12/2099,0,NaN,NaN
4O/IC-004,Montenegro,Istok Crne Gore,Kom vasojevićki,2461,8074,19.6677,42.6879,19.6677,42.6879,10,3,01/03/2019,31/12/2099,0,NaN,NaN


In [13]:
SOTA_summits_df.dtypes

AssociationName     string
RegionName          string
SummitName          string
AltM                 int32
AltFt                int32
GridRef1            string
GridRef2            string
Longitude          float64
Latitude           float64
Points               int32
BonusPoints          int32
ValidFrom           object
ValidTo             object
ActivationCount      int32
ActivationDate      object
ActivationCall      object
dtype: object

### Spots analysis

If you look at the spots_dfdataframe above, it may includes duplicates. The same activation may be spotted more than once. For visualisation purpose, I'll take the latest spot for operator-summit pair. The logic behind this is:
- it's highly unlikely to activate two summits by the same person within an hour,
- if two spots for the same operator-summit pair are exactly the same, the newest one is a confirmation "I'm still activating", so I'll take it to countdown time,
- if operator changed mode or band, the newest spot present the latest available information about the activation.

In [14]:
# drop duplicated activator-summit pairs from spots_df to avoid double visualisation for them
# only last spot sent by activator on a summit is considered

spots_df = spots_df.drop_duplicates(subset = ['activatorCallsign', 'summit'])
print(f'{len(spots_df)} found without duplicates.')

27 found without duplicates.


Now I'm going to prepare spots_df dataframe for visualisation by adding colums, where data from other dataframes (bands_df, modes_df and summits_df) will be copied. Temporary they will be filled with ```None``` values.
When it's done, spots_df dataframe will be reindexed to make sure indexes are consecutive numbers, therefore we'll avoid errors when looping throught its rows

In [15]:
# create empty columns for data required for visualisation (will b filled with database data)
# then re-index this dataframe

spots_df['longitude'] = None
spots_df['latitude'] = None
spots_df['points'] = None
spots_df['summitName'] = None
spots_df['mode_color'] = None
spots_df['band'] = None
spots_df['band_color'] = None
spots_df = spots_df.reset_index(drop = True)

C:\Users\jkacz\AppData\Local\Temp\ipykernel_16712\3895443260.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spots_df['longitude'] = None
C:\Users\jkacz\AppData\Local\Temp\ipykernel_16712\3895443260.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spots_df['latitude'] = None
C:\Users\jkacz\AppData\Local\Temp\ipykernel_16712\3895443260.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

The truth is that spots sometimes are sent with errors. For this analysis the most important is when summit code spotter is not included in SOTA Database. It may be a result of three situations:
- spotter's typo,
- spotter is not sure of summits references, but wants to forward the frequency to chasers,
- spotted summit has been added to SOTA database after last refresh of CSV files used.

To track if this happen in current list of spots, I'm creating a list to store errors found in next step.
Create a list to handle errors - misspelled codes or updated database.

In [16]:
# list to keep summit codes not found in SOTA Database file
# List of summits is periodically updated, but typos in summits codes in spots are also common
summits_errors = []

Now we are ready to match 
With a loop through all spots (hope you remember we've removed duplicates) script will check if summit reference is present in SOTA Databased. If no, summit will be added to errors list with no further action.

If summit is found, following data will be copied from SOTA database to spots_df DataFrame for each spot:
- longitude and latutide,
- number of SOTA points (from 1 to 10),
- name of the summit.

Also, based on spots data, time since spot's update (in hours), activation's mode and band are populated. For mode and band, declared color will be also assigned.

For wisualisation purpose, I'm also creating popup Series to store activation description to be displayed when marker is clicked on the map. Sample looks like this:

> Summit Red Mountain - W6/CT-133 (4 points) activated by KX6I on 14.061 - CW 3 minutes ago.


In [17]:
# copying relevant data for visualisation from SOTA database extract to spots dataframe
# also adding time since spot in hour fraction and description of spot
# adding previously defined colorcodes for band and mode to each spot, together with a band for each one
# popup column provides a summary of activation to be displayed on map
for i in range(0, len(spots_df)):
    # if summits data are correct,prepare spot's data for visualisation
    if spots_df.loc[i, ('summit')].upper() in SOTA_summits_df.index:
        spots_df.loc[i, ('longitude')] = SOTA_summits_df['Longitude'][spots_df.loc[i, 'summit']]
        spots_df.loc[i, ('latitude')] = SOTA_summits_df['Latitude'][spots_df.loc[i, 'summit']]
        spots_df.loc[i, ('points')] = SOTA_summits_df['Points'][spots_df.loc[i, 'summit']]
        spots_df.loc[i, ('summitName')] = SOTA_summits_df['SummitName'][spots_df.loc[i, 'summit']]
        spots_df.loc[i, ('time_since_spot')] = datetime.utcnow()-spots_df.loc[i, ('timeStamp')]
        spots_df.loc[i, ('time_since_spot')] = spots_df.loc[i, ('time_since_spot')]/timedelta(hours=1)
        spots_df.loc[i, ('popup')] = f"Summit {spots_df.loc[i, ('summitName')].title()} - {spots_df.loc[i, ('summit')]} ({spots_df.loc[i, ('points')]} points)\nactivated by {spots_df.loc[i, ('activatorCallsign')].upper()}\non {spots_df.loc[i, ('frequency')]} - {spots_df.loc[i, ('mode')].upper()}\n{round(spots_df.loc[i, ('time_since_spot')]*60)} minutes ago\n."
        spots_df.loc[i, ('mode')] = spots_df.loc[i, ('mode')].upper() # for correct match between spots and summits DataFrames
        for band in bands_df.index: # assess band based on frequency spotted
            if (spots_df.loc[i, ('frequency')] >= bands_df['lower_freq'][band]) and (spots_df.loc[i, ('frequency')] <= bands_df['upper_freq'][band]):
                spots_df.loc[i, ('band_color')] = bands_df['color'][band]
                spots_df.loc[i, ('band')] = band
        for j in modes_df.index:
            if spots_df.loc[i, ('mode')] == modes_df.iloc[j]['mode'].upper():
                spots_df.loc[i,('mode_color')] = modes_df.iloc[j]['color']
    # if summit isn't found in database, print warning, save it on a list and leave their data with None
    else:
        print(f"Summit {spots_df.loc[i, ('summit')]} activated by {spots_df.loc[i, ('activatorCallsign')].upper()} on {spots_df.loc[i, ('frequency')]} - {spots_df.loc[i, ('mode')].upper()}  NOT FOUND.")
        summits_errors.append({spots_df.loc[i, ('summit')]})

Summit FL/VL-XXX activated by TM15SOTA on 7.0315 - CW  NOT FOUND.


The spots_df DataFrame looks like this and is ready for visualisation.

In [18]:
spots_df.head()

,id,userID,timeStamp,comments,callsign,associationCode,summitCode,activatorCallsign,activatorName,frequency,...,summit,longitude,latitude,points,summitName,mode_color,band,band_color,time_since_spot,popup
0,658095,0,2022-04-23 10:26:02.797,[RBNHole] at DJ9IE 23 WPM 17 dB SNR,RBNHOLE,EI,IW-026,EI3KA/P,John,14.066,...,EI/IW-026,-9.86207,53.58478,6,Garraun,red,14 MHz,orange,0.019989,Summit Garraun - EI/IW-026 (6 points)\nactivat...
1,658094,0,2022-04-23 10:25:47.243,51 in Cardiff Kevin,GW5NIC,GW,MW-036,MW0KXN/P,Kevin,145.450,...,GW/MW-036,-4.17936,51.93484,1,Pen-crug-melyn,yellow,144 MHz,blue,0.024319,Summit Pen-Crug-Melyn - GW/MW-036 (1 points)\n...
2,658093,0,2022-04-23 10:24:37.007,[SOTA Spotter],CT2GSN,CT,MN-049,CT2GSN/P,João,28.461,...,CT/MN-049,-8.40365,41.41831,1,Santa Tecla,blue,28 MHz,green,0.043838,Summit Santa Tecla - CT/MN-049 (1 points)\nact...
3,658092,0,2022-04-23 10:24:33.923,None,MM7MWL,GM,ES-034,MM7MWL/P,Mike,14.273,...,GM/ES-034,-3.09886,57.13856,4,Mona Gowan,blue,14 MHz,orange,0.044703,Summit Mona Gowan - GM/ES-034 (4 points)\nacti...
4,658091,0,2022-04-23 10:20:05.233,[SOTA Spotter],SQ9RNW,SP,WS-002,SQ9RNW/P,MIROSLAW,28.450,...,SP/WS-002,18.1692,50.4558,1,Góra Świętej Anny,blue,28 MHz,green,0.119363,Summit Góra Świętej Anny - SP/WS-002 (1 points...


But before we move forward, I'll save incorrectly spotted rows in summits_errors.txt file. I'm using it to check if CSV file with summits database stored locally needs an update.

In [19]:
# save errors to file
if len(summits_errors) != 0:
    with open('summits_errors.txt', 'a') as f:
        for error in summits_errors:
            f.write(f'{error}\n')

## Visualisation of spots

In this script, spots will be visualised on the map with the use of Folium module. First, we'll create a map centered near Kraków (the city where I live) with the shaded terrain background.

In [20]:
# create a map in Folium - just works
activations_map = folium.Map(location = [50,20], # map is centered on Kraków - city where I live
                        tiles="Stamen Terrain", 
                        zoom_start=2 # show whole world at once
                            )

Now every spot kept in spots_df DataFrame will be visualised as a circle on the map with the use of CirleMarker markers. Each point provides following information:
- activated summit's **location** - circle's center;
- **time** since spot - circle's radius;
- activated **band** - circle's filling;
- **mode** use for activiation - circle's border.

Also, previously defined pop-up text is available when user clicks activation marker.

In [22]:
# add spots to a map
for i in range(0, len(spots_df)): # add point for every spot (with duplicates removed)
    if spots_df.loc[i, ('longitude')] != None:  # ignore spots where no reference data in SOTA database was found
        folium.CircleMarker(
        location = [spots_df.loc[i, ('latitude')], spots_df.loc[i, ('longitude')]], # spot's location)
        radius = (1 - spots_df.loc[i, ('time_since_spot')])*15, # radius is proportional to time from sending
                                                                # the spot. The newest spot, the larger circle
        popup = spots_df.loc[i, ("popup")],
        fill_color = spots_df.loc[i, ('band_color')], # circle's fill represents activation's band
        weight = 3,
        color = spots_df.loc[i, ('mode_color')], # border color represents activation's mode
        fill_opacity = 1
    ).add_to(activations_map)
    
display(activations_map)

Enjoy tracking current activations!